In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, precision_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from scipy.stats import uniform

In [2]:

# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_01/train_01/',
    './Documents/Research/can-train-and-test/set_02/train_01/',
    './Documents/Research/can-train-and-test/set_03/train_01/',
    './Documents/Research/can-train-and-test/set_04/train_01/'
]

# Test files for DoS attack
dos_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/DoS-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/DoS-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/DoS-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/DoS-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/DoS-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/DoS-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/DoS-2.csv'
]

# Test files for Force neutral attack
fn_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/force-neutral-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/force-neutral-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/force-neutral-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/force-neutral-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/force-neutral-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/force-neutral-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/force-neutral-2.csv'
]

# Test files for RPM attack
rpm_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/rpm-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/rpm-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-2.csv'
]

# Test files for standstill attack
standstill_files = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/standstill-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/standstill-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/standstill-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/standstill-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/standstill-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/standstill-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/standstill-2.csv'
]

In [3]:
# Function to Load Data
def load_data_from_directory(directory_path):
    data_frames = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            df = pd.read_csv(file_path)
            data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)

# Function to Preprocess Data
def hex_to_int(x):
    try:
        return int(str(x), 16)
    except ValueError:
        return 0  # Handle non-hexadecimal values
        
def preprocess_data(df):
    df['timestamp'] = pd.to_datetime(df['timestamp']).astype(np.int64) // 10**9
    df['arbitration_id'] = df['arbitration_id'].apply(hex_to_int)
    df['data_field'] = df['data_field'].apply(hex_to_int)
    return df

# Function to Extract Features and Labels
def extract_features_labels(df, label_col='attack'):
    X = df.drop(columns=label_col)
    y = df[label_col]
    return X, y

# Function to load and combine attack files
def load_combine_attack_files(files):
    data_frames = []
    for file in files:
        df = pd.read_csv(file)
        data_frames.append(df)
    combined_df = pd.concat(data_frames, ignore_index=True)
    return combined_df

In [4]:
# Function to Train Logistic Regression Model with RandomizedSearchCV
def train_logistic_regression(X_train, y_train):
   # Apply Random Under Sampling for Handling Class Imbalance
    rus = RandomUnderSampler(random_state=42)
    X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)


    # Standardize the Features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_resampled)

    # Define the model
    log_reg = LogisticRegression(solver='liblinear')

    # Define parameter distribution
    param_dist = {
        'C': uniform(0.01, 10),  # Regularization strength
        'penalty': ['l1', 'l2']  # Regularization type
    }

    # Set up RandomizedSearchCV
    random_search = RandomizedSearchCV(
        log_reg, param_distributions=param_dist, n_iter=300, scoring='f1', n_jobs=-1, cv=5, random_state=42
    )

    # Record training start time
    start_time = time.time()
    
    # Perform the random search
    random_search.fit(X_train_scaled, y_train_resampled)

    # Calculate training time
    training_time = time.time() - start_time
    
    return random_search.best_estimator_, scaler, training_time

# Function to Test the Model
def test_model(model, scaler, X_test, y_test):
    # Standardize the Features
    X_test_scaled = scaler.transform(X_test)

    # Record testing start time
    start_time = time.time()
    
    # Predict on Test Data
    test_predictions = model.predict(X_test_scaled)

     # Calculate testing time
    testing_time = time.time() - start_time
    
    # Evaluate Test Performance
    conf_matrix_test = confusion_matrix(y_test, test_predictions)
    mcc_test = matthews_corrcoef(y_test, test_predictions)
    accuracy_test = accuracy_score(y_test, test_predictions)
    precision_test = precision_score(y_test, test_predictions, average='weighted', zero_division=0)
    recall_test = recall_score(y_test, test_predictions, average='weighted', zero_division=0)
    f1_test = f1_score(y_test, test_predictions,  average='weighted', zero_division=0)
    informedness_test = recall_test - (1 - recall_test)
    markedness_test = precision_test - (1 - precision_test)

    return {
        'conf_matrix': conf_matrix_test,
        'mcc': mcc_test,
        'accuracy': accuracy_test,
        'precision': precision_test,
        'recall': recall_test,
        'f1_score': f1_test,
        'informedness': informedness_test,
        'markedness': markedness_test,
        'testing_time': testing_time
    }



In [5]:
# Load and preprocess training data
df_train = pd.concat([load_data_from_directory(train_dir) for train_dir in train_directories], ignore_index=True)
df_train = preprocess_data(df_train)
X_train, y_train = extract_features_labels(df_train)

In [6]:
# Train the Logistic Regression Model
best_model, scaler, training_time = train_logistic_regression(X_train, y_train)

In [7]:
# Load and preprocess testing data - DOS attack
df_test = pd.concat([pd.read_csv(file) for file in dos_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (DOS Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (DOS Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.24 seconds

-----------------------------------------------------------------------
Accuracy: 61.02%
Precision: 87.71%
Recall: 61.02%
F1-Score: 71.92%
Matthews Correlation Coefficient: -0.1569
Informedness: 0.2204
Markedness: 0.7543


In [8]:
# Load and preprocess testing data - Force Neutral
df_test = pd.concat([pd.read_csv(file) for file in fn_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Force neutral Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Force neutral Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.35 seconds

-----------------------------------------------------------------------
Accuracy: 64.64%
Precision: 99.77%
Recall: 64.64%
F1-Score: 78.45%
Matthews Correlation Coefficient: -0.0220
Informedness: 0.2928
Markedness: 0.9955


In [9]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in rpm_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (RPM Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (RPM Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.32 seconds

-----------------------------------------------------------------------
Accuracy: 64.48%
Precision: 99.78%
Recall: 64.48%
F1-Score: 78.30%
Matthews Correlation Coefficient: 0.0064
Informedness: 0.2896
Markedness: 0.9956


In [10]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in standstill_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Standstill Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Standstill Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.19 seconds

-----------------------------------------------------------------------
Accuracy: 64.18%
Precision: 99.41%
Recall: 64.18%
F1-Score: 78.00%
Matthews Correlation Coefficient: -0.0358
Informedness: 0.2836
Markedness: 0.9882


In [11]:
# Test files for DoS attack
double_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/double-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/double-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/double-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/double-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/double-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/double-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/double-2.csv'
]

# Test files for Force neutral attack
triple_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/triple-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/triple-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/triple-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/triple-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/triple-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/triple-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/triple-2.csv'
]

fuzzing_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/fuzzing-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/fuzzing-4.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/fuzzing-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/fuzzing-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/fuzzing-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/fuzzing-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/fuzzing-2.csv'
]

interval_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/interval-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/interval-4.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/interval-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/interval-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/interval-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/interval-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/interval-2.csv'
]

speed_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/speed-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/speed-4.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-2.csv'
]

systematic_files = [
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/systematic-3.csv',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/systematic-4.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/systematic-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/systematic-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/systematic-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/systematic-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/systematic-2.csv'
]

rpmaccessory_files = [
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/rpm-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/rpm-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/rpm-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/rpm-accessory-2.csv'
]

speedaccessory_files = [
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/speed-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/speed-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/speed-accessory-2.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-accessory-1.csv',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/speed-accessory-2.csv'
]

In [12]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in double_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Double Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Double Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.35 seconds

-----------------------------------------------------------------------
Accuracy: 63.88%
Precision: 99.17%
Recall: 63.88%
F1-Score: 77.65%
Matthews Correlation Coefficient: -0.0199
Informedness: 0.2775
Markedness: 0.9833


In [13]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in triple_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Triple Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Triple Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.28 seconds

-----------------------------------------------------------------------
Accuracy: 64.23%
Precision: 97.97%
Recall: 64.23%
F1-Score: 77.15%
Matthews Correlation Coefficient: 0.0429
Informedness: 0.2847
Markedness: 0.9594


In [14]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in fuzzing_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Fuzzing Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Fuzzing Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.21 seconds

-----------------------------------------------------------------------
Accuracy: 64.39%
Precision: 97.06%
Recall: 64.39%
F1-Score: 76.25%
Matthews Correlation Coefficient: 0.1370
Informedness: 0.2879
Markedness: 0.9412


In [15]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in interval_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Interval Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Interval Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.26 seconds

-----------------------------------------------------------------------
Accuracy: 63.28%
Precision: 95.62%
Recall: 63.28%
F1-Score: 76.10%
Matthews Correlation Coefficient: -0.0847
Informedness: 0.2657
Markedness: 0.9125


In [16]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in speed_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Speed Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Speed Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.54 seconds

-----------------------------------------------------------------------
Accuracy: 64.24%
Precision: 99.79%
Recall: 64.24%
F1-Score: 78.03%
Matthews Correlation Coefficient: 0.0522
Informedness: 0.2848
Markedness: 0.9957


In [17]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in systematic_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Systematic Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Systematic Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.19 seconds

-----------------------------------------------------------------------
Accuracy: 64.21%
Precision: 98.92%
Recall: 64.21%
F1-Score: 77.45%
Matthews Correlation Coefficient: 0.0734
Informedness: 0.2841
Markedness: 0.9784


In [18]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in rpmaccessory_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (RPM Accessory Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (RPM Accessory Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.02 seconds

-----------------------------------------------------------------------
Accuracy: 67.76%
Precision: 99.55%
Recall: 67.76%
F1-Score: 80.61%
Matthews Correlation Coefficient: -0.0107
Informedness: 0.3553
Markedness: 0.9910


In [19]:
# Load and preprocess testing data
df_test = pd.concat([pd.read_csv(file) for file in speedaccessory_files], ignore_index=True)
df_test = preprocess_data(df_test)
X_test, y_test = extract_features_labels(df_test)

# Test the Model
test_metrics = test_model(best_model, scaler, X_test, y_test)

# Print Combined Testing Metrics
print(f"\n===============================================================================")
print(f"\nTesting Metrics (Speed Accessory Attack Test Files):")
print(f"\n-----------------------------------------------------------------------")
print(f"\nTraining Time: {training_time:.2f} seconds")
print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
print(f"\n-----------------------------------------------------------------------")
print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
print(f"Informedness: {test_metrics['informedness']:.4f}")
print(f"Markedness: {test_metrics['markedness']:.4f}")



Testing Metrics (Speed Accessory Attack Test Files):

-----------------------------------------------------------------------

Training Time: 203.09 seconds
Testing Time: 0.04 seconds

-----------------------------------------------------------------------
Accuracy: 65.18%
Precision: 99.63%
Recall: 65.18%
F1-Score: 78.55%
Matthews Correlation Coefficient: 0.0783
Informedness: 0.3035
Markedness: 0.9926
